In [ ]:
import requests
import folium
import json
import pystac

In [ ]:
import geopandas as gpd
gf = gpd.read_file("https://raw.githubusercontent.com/scottyhq/snotel/main/snotel-sites.geojson")
all_fields = list(gf.drop(columns='geometry').columns)

In [ ]:
m = folium.Map(
    location=(47.6062, -122.3321),
    tiles="stamenterrain",
    zoom_start= 4,
    control_scale = True,
)

# Can also use geopandas explore() method here
# #mymarker = folium.Marker(icon=folium.Icon(color='green')) # Having trouble overriding styling...
mystyle = lambda x: {'color': 'cyan' if x['properties']['isActive']==True else 'magenta'}
mymarker = folium.Circle(fill=False, radius=200)
snotel_stations = "https://raw.githubusercontent.com/scottyhq/snotel/main/snotel-sites.geojson"
gjson = folium.GeoJson(snotel_stations, 
                       embed=False, 
                       marker=mymarker,
                       style_function=mystyle, 
                       name="snotel").add_to(m)

folium.features.GeoJsonTooltip(fields=["code"], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson)

# Add a Planetary Computer Layer
# Docs: https://planetarycomputer.microsoft.com/docs/quickstarts/using-the-data-api/

# Get this url from Planetary computer Explorer
item_url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc/items/S1A_IW_GRDH_1SDV_20230111T234137_20230111T234202_046743_059A8C_rtc"
# Add to Map (PC data API takes care of dynamic tiling)
item = pystac.Item.from_file(item_url)
r = requests.get(item.assets["tilejson"].href).json()
tiles = r["tiles"][0]
folium.TileLayer(tiles=tiles, overlay=True, name='PC RTC', attr="MS Planetary Computer").add_to(m)

# Add a public COG custom TIF layer using TiTiler (thanks devseed!)
tif = "https://sentinel-s1-rtc-indigo.s3.us-west-2.amazonaws.com/tiles/RTC/1/IW/12/S/YJ/2016/S1B_20161121_12SYJ_ASC/Gamma0_VV.tif"
tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
expression = "expression=sqrt(b1)&rescale=0,1"
virtual_tiles = f"{tiler}?url={tif}&{expression}"
folium.TileLayer(tiles=virtual_tiles, overlay=True, name='Indigo RTC', attr="IndigoAg").add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
m.save("vector-icons.html")